In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

#### Forward and backward passes

In [2]:
#export 

from exp.nb_01 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train, y_train, x_valid, y_valid))

def normalize(x, m, s): return (x-m)/s

In [3]:
x_train, y_train, x_valid, y_valid = get_data()

In [4]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std

(tensor(0.1304), tensor(0.3073))

Mean and std are not 0 and 1 but they are not - so we will normalize them

IMPORTANT:
We don't want the same for the validation set because then the two datasets would be on different scales, so we use the same mean and std for both the training and the validation sets!

In [5]:
x_train = normalize(x_train, train_mean, train_std)
x_valid = normalize(x_valid, train_mean, train_std)

In [6]:
train_mean, train_std = x_train.mean(),x_train.std()
train_mean, train_std

(tensor(3.0614e-05), tensor(1.))

In [7]:
valid_mean, valid_std = x_valid.mean(),x_valid.std()
valid_mean, valid_std

(tensor(-0.0058), tensor(0.9924))

In [3]:
#export
def test_near_zero(a,tol=1e-3): assert a.abs()<tol, f'Near zero: {a}'

In [9]:
test_near_zero(x_train.mean())
test_near_zero(1-x_train.std())

In [10]:
n, m  = x_train.shape
c = y_train.max()+1
n, m, c

(50000, 784, tensor(10))

In [11]:
math.sqrt(784)

28.0

##### Basic architecture

Simple neural net with a single hidden layer and one output activation instead of 10 so we can use MSE instead of cross-entropy

In [12]:
# num hidden 
nh = 50

In [13]:
# m is no. columns 768 x no.hidden layers
w1 = torch.randn(m,nh)
b1 = torch.zeros(nh)
w2 = torch.rand(nh,1)
b2 = torch.zeros(1)
# w/o dividing by sqrt

In [14]:
x_valid.mean(), x_valid.std()

(tensor(-0.0058), tensor(0.9924))

Why the kaiming initialization is helpful is demonstrated below, look at the mean and std of t's output and then look at it with the sqrt division

In [15]:
def lin(x,w,b): return x@w + b

In [16]:
t = lin(x_valid, w1, b1)

In [17]:
t.mean(), t.std()

(tensor(0.1967), tensor(27.1130))

In [18]:
# m is no. columns 768 x no.hidden layers
w1 = torch.randn(m,nh)/math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.rand(nh,1)/math.sqrt(nh)
b2 = torch.zeros(1)
# dividing by /math.sqrt is called a kaiming initialization

We want the second layer inputs to also be mean 0 and std 1

In [19]:
t = lin(x_valid, w1, b1)

In [20]:
t.mean(), t.std()

(tensor(0.0360), tensor(1.0157))

Much better...

This initialization makes a big difference when training neural networks

Fixup initialization https://arxiv.org/abs/1901.09321 allows for training of residual layers for networks up to 10,000 layers deep

In [21]:
# rectified linear unit function is only 1 line of code
# clamp_min replaces negative values with zero
def relu(x): return x.clamp_min(0.)

In [22]:
t = relu(lin(x_valid, w1, b1))

In [23]:
t.mean(), t.std()

(tensor(0.4209), tensor(0.6212))

Paper: Delving Deep into Rectifiers - Kaiming He et al.
https://arxiv.org/abs/1502.01852

This no longer has mean 0 and std 1 because relu cut all the negative values

It helps to read papers of competition winners like the ImageNet winner above

An alternative initialization is called Glorot initialization - a normalized initialization 2010 paper - yet this initialization fails to cope with the impact of the RELU

The trick to the He initialization is to double the numerator 

In [24]:
# m is no. columns 768 x no.hidden layers
w1 = torch.randn(m,nh)*math.sqrt(2/m)
# dividing by /math.sqrt is called a kaiming initialization

In [25]:
w1.mean(), w1.std()

(tensor(-0.0002), tensor(0.0507))

In [26]:
w1 = torch.randn(m,nh)*math.sqrt(2/m)
t1 = relu(lin(x_valid, w1, b1))
t1.mean(), t.std()

(tensor(0.5555), tensor(0.6212))

In [27]:
t1.mean(), t1.std()

(tensor(0.5555), tensor(0.8435))

In [28]:
def relu(x): return x.clamp_min(0.) - 0.5

This fucks up the mean though

Read 2.2 of Resnet paper

Note:
If you subtract 0.5 from the relu function - the mean tends to return to 0

In [29]:
#export 
from torch.nn import init

In [30]:
w1 = torch.zeros(m,nh)
init.kaiming_normal_(w1, mode='fan_out') # fan_out maintains unit_variance during backaward pass
t = relu(lin(x_valid, w1, b1))

In [31]:
t.mean(), t.std()

(tensor(0.0095), tensor(0.7625))

In [32]:
import torch.nn

In [33]:
w1.shape

torch.Size([784, 50])

In [34]:
torch.nn.Linear(m, nh).weight.shape

torch.Size([50, 784])

It's the other way around!

In [35]:
torch.nn.Linear.forward??

In [36]:
torch.nn.functional.linear??

#output = input.matmul(weight.t()) - uses transpose and flips the dims around

In [37]:
torch.nn.Conv2d??

In [38]:
torch.nn.modules.conv._ConvNd??
#     init.kaiming_uniform_(self.weight, a=math.sqrt(5)) still in the code and works poorly

In [39]:
torch.nn.modules.conv._ConvNd.reset_parameters??

In [40]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [41]:
%timeit -n 10 _=model(x_valid)

17.8 ms ± 4.14 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [42]:
assert model(x_valid).shape==torch.Size([x_valid.shape[0],1])

#### Loss function: MSE

In [43]:
model(x_valid).shape

torch.Size([10000, 1])

We want to remove a unit-axis by using squeeze

In [6]:
#export
# it helps to define which dimension you want to remove with squeeze 
# some batchsizes at the end of an epoch may only contain 1 value which gets turned into a scalar and tends to cause models to fail
def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean()

In [45]:
# converting to floats for MSE to work
y_train, y_valid = y_train.float(), y_valid.float()

In [46]:
preds = model(x_train)

In [47]:
preds.shape

torch.Size([50000, 1])

In [48]:
mse(preds, y_train)

tensor(28.9091)

Learn Matrix calculus here:
https://explained.ai/matrix-calculus/index.html

We want the gradient of the output with respect to the parameters

y_hat = mse(lin2(relu(lin1(x))),y)

Chain rule: We take the derivative of each element and multiply them together

#### Gradients and backward pass

In [49]:
def mse_grad(inp, targ):
    # grad of loss with respect to output of previous layer
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]

When looking at the Relu function plotted if x < 0 gradient is 0, if x > 0 gradient is 1

In [50]:
# chain rule requires us to multiply by out.g

def relu_grad(inp, out):
    # grad of relu with respect to input activations
    inp.g = (inp>0).float() * out.g

In [51]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    # gradient of a matrix product is the matrix product multiplied by the transpose
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

Backpropagation is basically the chainrule with a log of all the intermediate calculations so they don't need to be calculated again

In [52]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    # loss - which is never actually used because it's not required to calculate the gradients
    #loss = mse(out, targ)
    
    # backward pass:
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

In [53]:
#forward_and_backward(x_train, y_train)

In [54]:
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig = x_train.g.clone()

AttributeError: 'Tensor' object has no attribute 'g'

In [ ]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [55]:
def forward(inp, targ):
    # forward pass:
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    # no actual need for loss in backward pass
    return mse(out, targ)

In [ ]:
loss = forward(xt2, y_train)

In [ ]:
loss.backward()

#### Refactoring

In [ ]:
# @ 1h59

In [56]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)-0.5
        return self.out
    
    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g

In [57]:
class Lin():
    def __init__(self, w, b): self.w, self.b = w,b
    
    def __call__(self, inp):
        self.inp = inp
        self.out = inp@self.w + self.b
        return self.out
    
    # gradient of output with respect to the inputs, the weights and the biases
    
    def backward(self):
        # gradient of matrix product
        self.inp.g = self.out.g @ self.w.t()
        # 1s, -1s and 0s denote 
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)
        

In [4]:
def Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out

    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [59]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    # forward pass    
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    # backward pass
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [ ]:
w1.g, b1.g, w2.g, b2.g = [None]*4
model = Model(w1, b1, w2, b2)

In [ ]:
%time loss = model(x_train, y_train)

In [60]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)

In [61]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.) - 0.5
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [62]:
class Lin(Module):
    def __init__(self, w, b): self.w, self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = torch.einsum('bi,bj->ij', inp, out.g)
        self.b.g = out.g.sum(0)

In [63]:
class Mse(Module):
    def forward(self,inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

In [64]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)    
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()            

In [ ]:
w1.g, b1.g, w2.g, b2.g = [None]*4
model = Model()

In [ ]:
%time loss = model(x_train, y_train)

In [ ]:
%time model.backward()

#### without Einsum

In [65]:
class Lin(Module):
    def __init__(self, w, b): self.w, self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = inp.t() @ out.g # same as einsum
        self.b.g = out.g.sum(0)

In [ ]:
w1.g, b1.g, w2.g, b2.g = [None]*4
model = Model()

In [ ]:
%time loss = model(x_train, y_train)

In [ ]:
%time model.backward()

In [ ]:
#export
from torch import nn

In [ ]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out)]
        self.loss = mse
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x.squeeze(), targ)

In [ ]:
model = Model(m,nh, 1)

In [ ]:
%time loss = model(x_train, y_train)

In [ ]:
%time loss.backward()

Export

In [8]:
!python notebook2script.py 02_fully_connected.ipynb

Converted 02_fully_connected.ipynb to nb_02.py
